In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 8.2 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import pandas as pd
import time
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Запуск в headless-режиме
options.add_argument("--no-sandbox")  # Отключение sandbox
options.add_argument("--disable-dev-shm-usage")  # Решение для ограниченной памяти
options.add_argument("--disable-gpu")  # Отключение GPU
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(options=options)

In [4]:
#first check
try:
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.bbc.com/weather/293397")
    print(f"Title: {driver.title}")
except Exception as e:
    print(f"Ошибка: {e}")
finally:
    driver.quit()

Title: Tel Aviv - BBC Weather


In [6]:
driver = webdriver.Chrome(options=options)
driver.get("https://www.bbc.com/weather/293397")
time.sleep(5)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "wr-time-slot__inner"))
)

# lists for saving
time_slots = []
temperatures = []
precipitations = []
wind_speeds = []
humidities = []
details = []

# find buttons for more inf
time_slot_buttons = driver.find_elements(By.CLASS_NAME, "wr-time-slot__inner")

for button in time_slot_buttons:
    try:

        driver.execute_script("arguments[0].scrollIntoView();", button)

        button.click()
        time.sleep(2)

        # time
        time_element = button.find_element(By.CLASS_NAME, "wr-time-slot-primary__hours")
        time_slots.append(time_element.text.strip() + ":00")

        # temp
        temperature_element = button.find_element(By.CLASS_NAME, "wr-value--temperature--c")
        temperatures.append(temperature_element.text.strip())

        precipitation_elements = button.find_elements(By.CLASS_NAME, "wr-time-slot-primary__precipitation")
        if precipitation_elements:
            precipitations.append(precipitation_elements[0].text.strip())
        else:
            precipitations.append(None)

        wind_elements = button.find_elements(By.CLASS_NAME, "wr-value--windspeed--kph")
        if wind_elements:
            wind_speeds.append(wind_elements[0].text.strip())
        else:
            wind_speeds.append(None)

        humidity_elements = button.find_elements(By.XPATH, ".//dt[contains(text(), 'Humidity')]/following-sibling::dd")
        if humidity_elements:
            humidities.append(humidity_elements[0].text.strip())
        else:
            humidities.append(None)

        details_elements = button.find_elements(By.CLASS_NAME, "wr-time-slot-secondary__weather-type-text")
        if details_elements:
            details.append(details_elements[0].text.strip())
        else:
            details.append(None)

    except Exception as e:

        print(f"Errrors: {e}")
        time_slots.append(None)
        temperatures.append(None)
        precipitations.append(None)
        wind_speeds.append(None)
        humidities.append(None)
        details.append(None)


driver.quit()

# Data Frame
weather_data = pd.DataFrame({
    "Time": time_slots,
    "Temperature": temperatures,
    "Precipitation": precipitations,
    "Wind Speed (km/h)": wind_speeds,
    "Humidity": humidities,
    "Details": details,
})

weather_data.to_csv("weather_data_today.csv", index=False)

print(weather_data)

     Time Temperature                Precipitation Wind Speed (km/h) Humidity  \
0   19:00         18°  7%\nchance of precipitation                        66%   
1   20:00         18°  7%\nchance of precipitation                        68%   
2   21:00         17°  5%\nchance of precipitation                        71%   
3   22:00         17°  0%\nchance of precipitation                        74%   
4   23:00         16°  6%\nchance of precipitation                        75%   
5   00:00         15°  0%\nchance of precipitation                        80%   
6   01:00         15°  0%\nchance of precipitation                        84%   
7   02:00         15°  0%\nchance of precipitation                        89%   
8   03:00         14°  0%\nchance of precipitation                        90%   
9   04:00         14°  0%\nchance of precipitation                        88%   
10  05:00         14°  0%\nchance of precipitation                        88%   

                           

Decide to put it to a function

In [7]:
def scrape_weather_data(driver):

    time_slots = []
    temperatures = []
    precipitations = []
    wind_speeds = []
    humidities = []
    details = []

    time_slot_buttons = driver.find_elements(By.CLASS_NAME, "wr-time-slot__inner")

    for button in time_slot_buttons:
        try:
            driver.execute_script("arguments[0].scrollIntoView();", button)

            button.click()
            time.sleep(2)


            time_element = button.find_element(By.CLASS_NAME, "wr-time-slot-primary__hours")
            time_slots.append(time_element.text.strip() + ":00")


            temperature_element = button.find_element(By.CLASS_NAME, "wr-value--temperature--c")
            temperatures.append(temperature_element.text.strip())

            precipitation_elements = button.find_elements(By.CLASS_NAME, "wr-time-slot-primary__precipitation")
            if precipitation_elements:
                precipitations.append(precipitation_elements[0].text.strip())
            else:
                precipitations.append(None)


            wind_elements = button.find_elements(By.CLASS_NAME, "wr-value--windspeed--kph")
            if wind_elements:
                wind_speeds.append(wind_elements[0].text.strip())
            else:
                wind_speeds.append(None)

            humidity_elements = button.find_elements(By.XPATH, ".//dt[contains(text(), 'Humidity')]/following-sibling::dd")
            if humidity_elements:
                humidities.append(humidity_elements[0].text.strip())
            else:
                humidities.append(None)


            details_elements = button.find_elements(By.CLASS_NAME, "wr-time-slot-secondary__weather-type-text")
            if details_elements:
                details.append(details_elements[0].text.strip())
            else:
                details.append(None)

        except Exception as e:
            print(f"Errors: {e}")
            time_slots.append(None)
            temperatures.append(None)
            precipitations.append(None)
            wind_speeds.append(None)
            humidities.append(None)
            details.append(None)


    return pd.DataFrame({
        "Time": time_slots,
        "Temperature": temperatures,
        "Precipitation": precipitations,
        "Wind Speed (km/h)": wind_speeds,
        "Humidity": humidities,
        "Details": details,
    })

In [8]:
driver = webdriver.Chrome(options=options)
driver.get("https://www.bbc.com/weather/293397")
time.sleep(5)


all_weather_data = pd.DataFrame()

# Collect data each day cycle
for day in range(1, 14):  # ID buttons
    try:

        button_id = f"daylink-{day}"
        next_day_button = driver.find_element(By.ID, button_id)
        driver.execute_script("arguments[0].scrollIntoView();", next_day_button)
        next_day_button.click()
        time.sleep(5)


        daily_weather_data = scrape_weather_data(driver)


        daily_weather_data["Day"] = day


        all_weather_data = pd.concat([all_weather_data, daily_weather_data], ignore_index=True)

    except Exception:
        print(f"There is no button for day  {day} ")
        break

driver.quit()


all_weather_data.to_csv("weather_data_all_days.csv", index=False)


print(all_weather_data)

      Time Temperature                 Precipitation Wind Speed (km/h)  \
0    06:00         14°   0%\nchance of precipitation                     
1    07:00         14°   0%\nchance of precipitation                     
2    08:00         15°   0%\nchance of precipitation                     
3    09:00         16°   0%\nchance of precipitation                     
4    10:00         18°   0%\nchance of precipitation                     
..     ...         ...                           ...               ...   
307  01:00         12°  16%\nchance of precipitation                     
308  02:00         12°  18%\nchance of precipitation                     
309  03:00         11°   0%\nchance of precipitation                     
310  04:00         11°   0%\nchance of precipitation                     
311  05:00         10°   0%\nchance of precipitation                     

    Humidity                              Details  Day  
0        86%          A clear sky and light winds    1

In [9]:
all_weather_data

,Time,Temperature,Precipitation,Wind Speed (km/h),Humidity,Details,Day
0,06:00,14°,0%\nchance of precipitation,,86%,A clear sky and light winds,1
1,07:00,14°,0%\nchance of precipitation,,85%,Sunny and light winds,1
2,08:00,15°,0%\nchance of precipitation,,81%,Sunny and light winds,1
3,09:00,16°,0%\nchance of precipitation,,76%,Sunny and a gentle breeze,1
4,10:00,18°,0%\nchance of precipitation,,71%,Sunny intervals and a gentle breeze,1
...,...,...,...,...,...,...,...
307,01:00,12°,16%\nchance of precipitation,,74%,A clear sky and light winds,13
308,02:00,12°,18%\nchance of precipitation,,73%,Partly cloudy and light winds,13
309,03:00,11°,0%\nchance of precipitation,,76%,Partly cloudy and light winds,13
310,04:00,11°,0%\nchance of precipitation,,78%,Partly cloudy and light winds,13


In [12]:
weather_data["Day"] = 1

In [13]:
combined_weather_data = pd.concat([weather_data, all_weather_data], ignore_index=True)

In [14]:
сombined_weather_data = combined_weather_data.sort_values(by="Day", ascending=True).reset_index(drop=True)

In [15]:
combined_weather_data

,Time,Temperature,Precipitation,Wind Speed (km/h),Humidity,Details,Day
0,19:00,18°,7%\nchance of precipitation,,66%,A clear sky and a gentle breeze,1
1,20:00,18°,7%\nchance of precipitation,,68%,A clear sky and light winds,1
2,21:00,17°,5%\nchance of precipitation,,71%,A clear sky and light winds,1
3,22:00,17°,0%\nchance of precipitation,,74%,A clear sky and light winds,1
4,23:00,16°,6%\nchance of precipitation,,75%,A clear sky and light winds,1
...,...,...,...,...,...,...,...
318,01:00,12°,16%\nchance of precipitation,,74%,A clear sky and light winds,13
319,02:00,12°,18%\nchance of precipitation,,73%,Partly cloudy and light winds,13
320,03:00,11°,0%\nchance of precipitation,,76%,Partly cloudy and light winds,13
321,04:00,11°,0%\nchance of precipitation,,78%,Partly cloudy and light winds,13


In [16]:
combined_weather_data.to_csv("all_days_weather_data.csv", index=False)

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
combined_weather_data['Datetime'] = pd.to_datetime(
    '2024-12-20' + ' ' + combined_weather_data['Time'],
    format='%Y-%m-%d %H:%M',
    errors='coerce'
)

In [23]:
combined_weather_data['Full_Date'] = pd.to_datetime(
    '2024-12-20', format='%Y-%m-%d') + pd.to_timedelta(combined_weather_data['Day'] - 1, unit='D')

combined_weather_data['Datetime'] = pd.to_datetime(
    combined_weather_data['Full_Date'].dt.strftime('%Y-%m-%d') + ' ' + combined_weather_data['Time'],
    errors='coerce')

In [24]:
filtered_data = combined_weather_data[combined_weather_data['Full_Date'] >= '2024-12-20']

In [27]:
import plotly.express as px

In [ ]:
filtered_data['Temperature'] = filtered_data['Temperature'].str.replace('°', '').astype(int)


In [31]:

fig = px.line(
    filtered_data,
    x="Datetime",
    y="Temperature",
    color="Day",
    title="Temperature from с 20.12.2024",
    labels={
        "Datetime": "Date and time",
        "Temperature": "Temperature (°C)",
        "Day": "Day"
    },
    markers=True
)


fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    legend_title="Day",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template="plotly_white"
)

fig.show()

In [33]:
mean_data = filtered_data.groupby("Day").agg(
    Mean_Temperature=("Temperature", "mean"),
    Mean_Humidity=("Humidity", lambda x: pd.to_numeric(x.str.replace('%', ''), errors='coerce').mean()),
    Mean_Precipitation=("Precipitation", lambda x: pd.to_numeric(x.str.replace('%\nchance of precipitation', ''), errors='coerce').mean())
).reset_index()

mean_data["Date"] = pd.date_range(start="2024-12-20", periods=len(mean_data))

In [35]:
mean_data


,Day,Mean_Temperature,Mean_Humidity,Mean_Precipitation,Date
0,1,16.400000,77.657143,10.142857,2024-12-20
1,2,16.291667,76.875000,10.458333,2024-12-21
2,3,15.791667,77.500000,39.166667,2024-12-22
3,4,14.916667,80.000000,5.875000,2024-12-23
4,5,15.166667,66.958333,0.000000,2024-12-24
5,6,15.250000,61.000000,4.041667,2024-12-25
6,7,14.041667,65.375000,0.666667,2024-12-26
7,8,14.500000,67.291667,9.625000,2024-12-27
8,9,14.458333,77.750000,45.333333,2024-12-28
9,10,14.166667,79.041667,14.458333,2024-12-29


In [37]:
fig = px.line(
    mean_data,
    x="Date",
    y=["Mean_Temperature", "Mean_Humidity", "Mean_Precipitation"],
    title="Average Temperature, Humidity, and Precipitation from 20.12.2024",
    labels={
        "Date": "Date",
        "value": "Value",
        "variable": "Variable"
    },
    markers=True
)

fig.layout.yaxis.tickformat = ',.1f'
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Variable",
    xaxis=dict(showgrid=True),
)
fig.show()

The chart shows the daily average temperature, humidity, and precipitation from December 20, 2024.

Temperature stays consistent, around 14-16°C, with slight fluctuations.

Humidity is mostly high, peaking around 80% but decreases slightly on some days
.
Precipitation varies greatly, with some days having almost no rain and others reaching over 40%.

The weather is generally cool and humid, with occasional rain.